# VGG16

### Import Required Libraries

In [1]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
import numpy as np
import splitfolders
from tensorflow.keras.applications import vgg16
from keras.applications.vgg16 import VGG16

### Set Hyperparameters and Paths

In [2]:
SEED = 42
IMG_HEIGHT, IMG_WIDTH = (224, 224)
BATCH_SIZE = 64
EPOCHS = 5
output=r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split'
OUTPUT_DIR = output
train_data_dir = f"{OUTPUT_DIR}/Balanced"
valid_data_dir = f"{OUTPUT_DIR}/val"
test_data_dir = f"{OUTPUT_DIR}/test"

### Load the Data

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical")


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
    class_mode="categorical")

Found 1925 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
Found 106 images belonging to 5 classes.


### Load the Model

In [13]:
base_model = VGG16(include_top=False, weights="imagenet")

#### Add the Layers

In [21]:
x = base_model.output
# print(x.shape)
x = GlobalAveragePooling2D()(x)
# print(x.shape)
x = Dense(1024, activation="softmax")(x)
# print(x.shape)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)
# print(predictions.shape)
model = Model(inputs=base_model.input, outputs=predictions)

### Model Summary

In [14]:
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

### Make the Trainable Layers False

In [15]:
for layer in base_model.layers:
    layer.trainable = False

### Compile the model by selecting Appropriate Optimizer

In [16]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"])

### Train the model

In [17]:
history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=5)

Epoch 1/5
31/31 [==============================] - 652s 21s/step - loss: 1.4620 - accuracy: 0.7112 - val_loss: 0.7379 - val_accuracy: 0.8300
Epoch 2/5
31/31 [==============================] - 903s 29s/step - loss: 0.1526 - accuracy: 0.9522 - val_loss: 0.4968 - val_accuracy: 0.8500
Epoch 3/5
31/31 [==============================] - 703s 23s/step - loss: 0.0574 - accuracy: 0.9834 - val_loss: 0.3986 - val_accuracy: 0.8800
Epoch 4/5
31/31 [==============================] - 514s 17s/step - loss: 0.0245 - accuracy: 0.9964 - val_loss: 0.3992 - val_accuracy: 0.8500
Epoch 5/5
31/31 [==============================] - 644s 21s/step - loss: 0.0120 - accuracy: 0.9995 - val_loss: 0.4076 - val_accuracy: 0.8400


### Evaluate the Test Accuracy

In [19]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(test_acc)

106/106 - 39s - loss: 0.3927 - accuracy: 0.8491 - 39s/epoch - 365ms/step
0.849056601524353


### Save the Vgg16 model

In [20]:
path=r"C:\Users\GREESHMANTH\Desktop\Major Project\Models\VGG16.h5"
model.save(path)